# 2023_05_22


### data cleaninng

traffic data의 city에는 미국의 모든 city가 들어 있어서  이름이 중복되는 경우가 존재함    
(아래 map을 보면 4 구역의 Long Beach가 나옴)  

우리가 원하는거는 Long Beach시/군 에 대한 데이터가 필요한데 traffic data에서는  Long Beach 동도 같이 city에 들어가 있는 느낌  


In [2]:
from IPython.display import IFrame

IFrame('../Meeting_Img/Long%20Beach_5_old.html', width=1200, height=800)

이를 해결하기 위해서 미국의 cences 데이터를 이용해서 우리가 원하는 Los Angles 밑에 있는 Long Beach만 선택해서 데이터 생성    
cences의 geohash와 traffic data의 geohash가 일치하는 경우에만 사용  
이를 모든 도시에 적용  

> 다음의 파일을 수정 : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/DataProcessing/GenerateData_Traffic.ipynb


이후에 기존의 html 파일과 비교해서 정상적으로 데이터가 생성되었는지 확인  

In [3]:
IFrame('../Traffic_Accient/GeoHash_html/Long Beach_5.html', width=1200, height=800)

### 데이터 생성 순서

1. EventId별 사고 데이터 전처리 
> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/DataProcessing/GenerateData_Traffic.ipynb 

2. 날씨 데이터 전처리
> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/DataProcessing/GenerateData_Weather.ipynb

3. 사고 데이터 및 날씨 데이터 병합 
> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/DataProcessing/GenerateData_Merge_TrafficWeather.ipynb

4. 결측치 추가 
> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/DataProcessing/GenerateData_insertIndex.ipynb

###  데이터 리뷰

* target값에 대한 데이터 리뷰  
발생빈도 , 기간별 통계치 및 geohash별 발생 건수 확인  


> Long Beach 5 HTML : http://202.31.200.70:8888/tree/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/DataProcessing/DataReview_HTML/Long%20Beach_5

In [1]:
import os
from IPython.display import IFrame

folder_path = '../Traffic_Accient/DataProcessing/DataReview_HTML/Long Beach_5'
file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith('.html'):
        file_path = os.path.join(folder_path, file_name)
        display(IFrame(file_path, width="100%", height="500px"))


* dataset review

dataset에 생성한 train data가 어떻게 들어가는지 확인인 및 weather data의 interpoloation 확인 

### params tuning

Long Beach 5를 기준으로 학습 진행  
성능이 잘 안나오는 관계로 hparmas tuning을 진행  

lr을 바꿔도 성능은 그대로 예측을 하지 못하고 있음   

> lr : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/BaseModel_Long%20Beach_5_lr.ipynb


batch_size가 너무 커서 학습을 못하고 있는것 같아서 batch_size를 조절    
시간이 너무 오래 걸리는 관계로 backend 서버 이용  

> batch_size: http://202.31.200.12:10382/notebooks/thkim/SEO/Emergency_Demand/BaseModel_Long%20Beach_5_batch_size.ipynb

터미널을 이용해서 mulit gpu 사용코드   
pytorch 버전은 1.0.0이 되면서 기존의 코드를 약간 수정    
pytorch-lighting을 ligthing으로 수정  

```python
# save_as_script.py
import argparse
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss

lr = 0.005
hidden = 64
atten_head = 4
dropout = 0.1

def main(batch_size):
    data = pd.read_csv('Long Beach_5.csv')

    max_prediction_length = 24
    max_encoder_length = 24*7
    training_cutoff = data["time_idx"].max() - max_prediction_length

    training = TimeSeriesDataSet(
        data[lambda x : x.time_idx <= training_cutoff],
        time_idx = "time_idx",
        target = "Accient",
        group_ids = ['GeoHash'],
        min_encoder_length=max_prediction_length//2,  # keep encoder length long (as it is in the validation set)
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        static_categoricals=["GeoHash"],
        time_varying_known_categoricals=[],
        time_varying_known_reals=[],
        #time_varying_unknown_categoricals=,
        time_varying_unknown_reals=data.columns[4: -2].tolist(), 
        target_normalizer=GroupNormalizer(
            groups=["GeoHash"], 
            transformation="relu",
            center = False
        ),
        add_relative_time_idx=True,
        add_target_scales=False,
        add_encoder_length=True,
        #allow_missing=True,
        allow_missing_timesteps = True,
        #predict_mode = False
    )

    validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=3)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=3)

    logger = TensorBoardLogger("Long Baech_logs")
    trainer = pl.Trainer(
        max_epochs=50,
        strategy="ddp",
        #accelerator='gpu', 
        devices=[0,1,2],
        gradient_clip_val=0.1,
        #detect_anomaly=True,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=lr,
        hidden_size=hidden ,
        attention_head_size=atten_head,
        dropout=dropout,  
        hidden_continuous_size=hidden,
        output_size=7,  
        loss=QuantileLoss(),
        reduce_on_plateau_patience=4,
    )

    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader
    )

    print(trainer.checkpoint_callback.best_model_path)

    raw_predictions, x , idx_df = tft.predict(val_dataloader, mode="raw", return_x=True , return_index=True)

    fig, axs = plt.subplots(nrows=raw_predictions['prediction'].shape[0], figsize=(10, raw_predictions['prediction'].shape[0]*5))

    for idx in range(raw_predictions['prediction'].shape[0]):  # plot examples
        tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True, ax=axs[idx])

    plt.savefig(f"batch_size_{batch_size}figure.png")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='PyTorch Forecasting Script')
    parser.add_argument('--batch_size', type=int, default=8, help='Batch size for training (default: 8)')
    args = parser.parse_args()

    main(args.batch_size)
```